In [154]:
   !pip install --upgrade "huggingface-hub>=0.30.0" "langchain-huggingface"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [156]:
import os
os.chdir("/Users/marouaoukrid/Desktop/MedicalChatbot/MedicalChatbot")

In [157]:
%pwd

'/Users/marouaoukrid/Desktop/MedicalChatbot/MedicalChatbot'

In [158]:
# Fixed imports - using langchain_community and langchain_huggingface
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

### Extract data from the pdf file

In [159]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [160]:
extracted_data=load_pdf_file(data='Data/')

In [161]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    texts_ = text_splitter.split_documents(extracted_data)

    return texts_

In [162]:
text_chunks = text_split(extracted_data=extracted_data)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 3426


In [34]:
#text_chunks

In [163]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',)
    return embeddings

In [164]:
embeddings = download_huggingface_embeddings()

In [165]:
query_results = embeddings.embed_query("Hello world")
print("Length:" ,len(query_results))

Length: 384


In [18]:
%pip install pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached pinecone_plugin_assistant-1.7.0-py3-none-any.whl.metadata (28 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)
Using cached pinecone_plugin_assistant-1.7.0-py3-none-any.whl (239 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pinecone]2/3 [pinecone]
Note: you may need to restart the kernel to use updated packages.


In [166]:

load_dotenv()

True

In [167]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
HUGGINGFACE_API_KEY = os.environ.get("HUGGINGFACE_API_KEY")

In [169]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)


index_name = "medibot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1")
    )

In [170]:

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY

In [171]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name,
    
)

In [191]:
llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=HUGGINGFACE_API_KEY,
    task="text-generation",
    temperature=0.4,
    max_new_tokens=500
)

In [173]:
docsearch

In [174]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [175]:
retrieved_docs = docsearch.similarity_search("What is Acne?")
retrieved_docs

[Document(id='c41dc95d-d4d0-4367-ae24-39ce57d954eb', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='The goal of treating moderate acne is to decrease\ninflammation and prevent new comedone formation. One\neffective treatment is topical tretinoin along with a topical\nGALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='f8d19d33-97e5-4570-9af9-60d52e38d3e1', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label

In [186]:
# Replace your current LLM setup with this:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="microsoft/DialoGPT-medium",
    model_kwargs={"temperature": 0.4, "max_new_tokens": 500},
    huggingfacehub_api_token=HUGGINGFACE_API_KEY
)

In [182]:

system_prompt = (
    "You are an assistant for question-answering tasks."
    "You're a medical expert."
    "Use the following pieces of retrieved context to answer the question.If you don't know the answer, just say that you don't know. Don't try to make up an answer."
    "Use three sentences maximum to answer the question."
    "Keet the answer concise and to the point."
    "\n\n"
    "{context}")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
        
    ]
)


In [183]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [192]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response['answer'])

HfHubHTTPError: 404 Client Error: Not Found for url: https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta (Request ID: Root=1-687456fb-684624495f4667ba5245f5f6;fe3b7db3-b0e9-43fd-9e5f-05ce3fe4201e)

from langchain